In [ ]:
from multiprocessing import Process, Queue
import re
import subprocess

from jakenode.database_connector import run_query

WORKFLOW_ID = 1

def extract_ip_address():
    ifconfig = subprocess.check_output('ifconfig eth0', shell=True).decode()
    extracted_text = re.findall(r'\Winet ([0-9]{3}\.[0-9]{2}(?:\.[0-9]{3}){2})\W', ifconfig)
    return extracted_text[0]

def run_xpra_window(queue, workflow_id=WORKFLOW_ID):
    script_target = 'run_two_window_app.py --account-id 1 --workflow-category-id 2'
    x11_screen = 80
    xpra_target_port = '8080'
    user = 'myuser'
    
    current_working_directory = subprocess.check_output('pwd')
    current_working_directory = current_working_directory.decode().strip()
    script_filepath = f'{current_working_directory}/{script_target}'

    base_command = f'xpra start :{x11_screen}'
    flags = [
        f'--bind-tcp=0.0.0.0:{xpra_target_port}',
        '--mdns=no',
        '--webcam=no',
        '--no-daemon',
        '--pulseaudio=no',
        '--min-quality=90',
        f'--start="python3 {script_filepath}"'
    ]
    
    flags = ' '.join(flags)
     
    command = f"su myuser -c '{base_command} {flags}'"
    print(command)
    subprocess.call(command, shell=True)
    return extract_ip_address()

run_xpra_window(None)

In [ ]:
!flask run --port 61000


In [ ]:
import subprocess


extract_ip_address()

In [ ]:
!apt-get install -y net-tools

In [ ]:
from itertools import chain

from jakenode.database_connector import run_query

def invert_address_types(address_types):
    column_types = {}
    for address_type, column_list in address_types.items():
        for column_name in column_list:
            column_types[column_name] = address_type
    return column_types

def generate_fetch_query(columns, destination_server):
    address_select_clause = f",\n            ".join(columns)

    return f"""
        SELECT
            {address_select_clause}
        FROM workflow_routes
        WHERE
            app_server_address='{destination_server}'
            AND active = 'TRUE'
    """
    
def fetch_taken_addresses(column_types, destination_server):
            
    query = generate_fetch_query(column_types.keys(), destination_server)
    
    taken_addresses = run_query(query, return_data_format=dict)
    filtered_taken_addresses = {}
    for column_name, address_type in column_types.items():
        filtered_taken_addresses.setdefault(address_type, [])
        filtered_taken_addresses[address_type] += [i for i in taken_addresses[column_name] if isinstance(i, int)]

    return filtered_taken_addresses

def assign_next_port(workflow_id):
    DESTINATION_SERVER = 'localhost'
    
    address_search_ranges = {
        'ports': (49152, 65535),
        'websockets': (49152, 65535),
        'x11_displays': (0, 99)
    }
    
    address_types = {
        'ports': ['xpra_port', 'info_panel_port'],
        'websockets': ['websocket'],
        'x11_displays': ['x11_display']
    }
    
    column_types = invert_address_types(address_types)

    workflow_addresses = run_query(
        f"""
            {generate_fetch_query(chain(*address_types.values()), DESTINATION_SERVER)}
            AND workflow_id = {workflow_id}
        """,
        return_data_format=dict
    )

    taken_address_dict = fetch_taken_addresses(column_types, DESTINATION_SERVER)
    update_column_dict = {}
    
    if not list(chain(*workflow_addresses.values())):
        # If no values are returned by query, do this:
        for column_name in workflow_addresses:
            address_type = column_types[column_name]
            taken_addresses = taken_address_dict[address_type]
            current_address, max_allowable_address = address_search_ranges[address_type]
            assigned_address = None

            max_value_err_msg = f'Cannot assign {address_type}.'
            max_value_err_msg += f'All values {current_address}-{max_allowable_address} already taken.'

            while assigned_address is None:
                if current_address > max_allowable_address:
                    raise Exception(max_value_err_msg)
                elif current_address not in taken_addresses:
                    assigned_address = current_address
                    taken_address_dict[address_type].append(assigned_address)
                else:
                    current_address += 1
            update_column_dict[column_name] = assigned_address
        return update_column_dict
        

#     create_new_row = False
#     assign_new_addresses = False
#     for column_name in column_types:
#         if not workflow_addresses.get(column_name):
#             if not create_new_row:
#                 create_new_row = True
#             else:
#                 create_new_row = False

#             assign_new_addresses = True
#         elif not workflow_addresses.get(column_name)[0]:
#             create_new_row = False
#             assign_new_addresses = True
#         else:
#             create_new_row = False
#             assign_new_addresses = False
        
#         if create_new_row:
#             run_query(
#                 """
#                     INSERT INTO workflow_routes (id, account_id, workflow_id, active)
#                     VALUES (
#                         (SELECT MAX(id) + 1 FROM workflow_routes),
#                         (SELECT account_id FROM workflows WHERE id = ?),
#                         ?,
#                         'TRUE'
#                     )
#                 """,
#                 sql_parameters=[workflow_id, workflow_id],
#                 commit=True
#             )

#         if assign_new_addresses:
#             address_type = column_types[column_name]
#             taken_addresses = taken_address_dict[address_type]
#             current_address, max_allowable_address = address_search_ranges[address_type]
#             assigned_address = None
            
#             max_value_err_msg = f'Cannot assign {address_type}.'
#             max_value_err_msg += f'All values {current_address}-{max_allowable_address} already taken.'
            
#             while assigned_address is None:
#                 if current_address > max_allowable_address:
#                     raise Exception(max_value_err_msg)
#                 elif current_address not in taken_addresses:
#                     assigned_address = current_address
#                 else:
#                     current_address += 1
#             update_column_dict[column_name] = assigned_address
            

#     if update_column_dict:
#         set_clause = []
#         for column_name, set_value in update_column_dict.items():
#             set_clause.append(f'    {column_name}={set_value}')
#         set_clause = ',\n            '.join(set_clause)
#         update_statement = f"""
#             UPDATE workflow_routes
#             SET
#             {set_clause},
#                 active='TRUE'
#             WHERE workflow_id=?
#             ;
#         """
#         run_query(update_statement, sql_parameters=[workflow_id], commit=True)
#         print(update_statement)

assign_next_port(10)

In [ ]:
from jakenode.database_connector import run_query
run_query("""
    SELECT * FROM workflow_routes
""")

In [ ]:
# !flask run -p 49151
!python3 app.py

In [ ]:
!su myuser -c 'xpra start :80 --bind-tcp=0.0.0.0:8080 --mdns=no --webcam=no --pulseaudio=no --min-quality=90 --start="python3 /srv/node_app/run_two_window_app.py --account-id 1 --workflow-category-id 2 --workflow-id 2"'

In [ ]:
!su myuser -c 'python3 run_two_window_app.py'
# su myuser -c 'xpra start :80 --bind-tcp=0.0.0.0:8080 --mdns=no --webcam=no --pulseaudio=no --min-quality=90 --start="python3 /srv/node_app/run_two_window_app.py --account-id 1 --workflow-category-id 2"'
# su myuser -c 'xpra start :80 --bind-tcp=0.0.0.0:8080 --mdns=no --webcam=no --pulseaudio=no --min-quality=90 --start="python3 /srv/node_app/run_two_window_app.py --account-id 1 --workflow-category-id 2"'